# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [3]:
# import required libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime

Next, import `Orders` from Ironhack's database into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [4]:
# your code here
driver = "mysql+pymysql"
user = "ironhacker_read"
password = "ir0nhack3r"
ip = "35.239.232.23"
database = "orders"

connection_string = f"{driver}://{user}:{password}@{ip}/{database}"
    
engine = create_engine(connection_string)

query = """
        SELECT * FROM orders
"""

orders = pd.read_sql(query, engine)

orders.to_csv("./orders.csv")
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397924 entries, 0 to 397923
Data columns (total 14 columns):
index           397924 non-null int64
InvoiceNo       397924 non-null int64
StockCode       397924 non-null object
year            397924 non-null int64
month           397924 non-null int64
day             397924 non-null int64
hour            397924 non-null int64
Description     397924 non-null object
Quantity        397924 non-null int64
InvoiceDate     397924 non-null object
UnitPrice       397924 non-null float64
CustomerID      397924 non-null int64
Country         397924 non-null object
amount_spent    397924 non-null float64
dtypes: float64(2), int64(8), object(4)
memory usage: 42.5+ MB


In [5]:
# drop double index + change types accordingly
orders['InvoiceDate']= pd.to_datetime(orders['InvoiceDate'])

---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [15]:
# your code here
# aggregate
orders_spend_sum = orders.groupby(["CustomerID"]).sum()

# quartiles
Q75_preferred = orders_spend_sum["amount_spent"].quantile(0.75)
Q95_VIP = orders_spend_sum["amount_spent"].quantile(0.95)

# VIPS
def customer_trans (x):
    if x >= Q95_VIP:
        return "VIP"
    elif x >= Q75_preferred:
        return "Preferred"
    else:
        return "Normal"
#transformation = lambda x: (x >= percentil95 and "VIP") or (x < percentil75 and "Normal") or "Preferred")
orders_spend_sum["customer_type"] = orders_spend_sum["amount_spent"].apply(lambda x: customer_trans(x))
orders_spend_sum.head()

,index,InvoiceNo,year,month,day,hour,Quantity,UnitPrice,amount_spent,customer_type
CustomerID,,,,,,,,,,
12346,37126,541431,2011,1,2,10,74215,1.04,77183.60,VIP
12347,30294108,101296926,365971,1383,441,2219,2458,481.21,4310.00,Preferred
12348,1900064,16869685,62324,257,111,472,2341,178.71,1797.24,Preferred
12349,26148016,42165457,146803,803,73,657,631,605.10,1757.55,Preferred
12350,838678,9231629,34187,34,51,272,197,65.30,334.40,Normal


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [16]:
# your code here - not completed - no time
orders_spend_sum["customer_type"].value_counts()

# needs merge to merge with order table to see ountry of custoemer ID - got removed as we did a sum grouped by Customer ID and country cannot be summed

#orders_customer = pd.DataFrame(orders.groupby(["customer_type", "Country"])["CustomerID"])
#orders_customer.loc["VIP"]

Normal       3254
Preferred     868
VIP           217
Name: customer_type, dtype: int64

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [8]:
# your code here